.

#### 1. 개요

타계정간 s3 Replication 기능 테스트

#### 2. s3 Replication 기능 요약

![3](https://user-images.githubusercontent.com/41605276/79062783-05c81400-7cd8-11ea-8447-6c51c072a3a4.png)

위에 그림과 같은 아키텍처에서 타계정 s3 간에 데이터 복제가 가능한지 궁금할 수도 있는데 관련해서 사용할 수 있는 옵션이 s3 Replication이라는 기능이 있다.

말그대로 s3 데이터를 복제해주는 기능이다. 
Source 버킷 콘솔에서 '복제규칙' 을 지정해주면 작동하는 기능이다.
Destination 등 설정값을 지정하고 활성화를 하면 그 시점부터 새로 변동되는 파일에 대해 replication을 해준다.
(기존에 저장되어 있던 파일들까지 반영하고자 한다면 S3 Sync 명령 등을 활용하여 사전에 source와 destination을 동기화하고, s3 replication을 적용하면 된다.)

AWS 공식문서는 아래의 URL을 참고하면 된다.

- 공식 도큐먼트 : https://docs.aws.amazon.com/ko_kr/AmazonS3/latest/user-guide/enable-replication.html#enable-replication-cross-account-destination


- AWS 블로그 참고자료 : https://aws.amazon.com/ko/blogs/korea/amazon-s3-introduces-same-region-replication/


원하는 폴더&파일만 복제가 가능하도록 '접두사 지정'이 가능하다.


#### 5.  [참고사항] S3 Replication 기능 테스트

- 위에 그림과 같은 아키텍처를 실제로 구현해보는 테스트

[가이드]

STEP 1) Source 및 Destination Bucket에 versioning 설정

![1](https://user-images.githubusercontent.com/41605276/79062977-96ebba80-7cd9-11ea-9794-aaf9ea6dcd27.PNG)

STEP 2) Source Bucket에 접속해서 아래와 같이 Replication Rule 설정

![4](https://user-images.githubusercontent.com/41605276/79063539-46765c00-7cdd-11ea-8b7e-be9f73f760ff.png)

* 참고로 위에 그림과 같이 replication 설정 시 IAM Role을 새로 생성한다고 지정하면 아래와 같이 생성된다.

IAM Role : s3crr_for_pms-s3synctest-bucket_to_minman2

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:Get*",
                "s3:ListBucket"
            ],
            "Effect": "Allow",
            "Resource": [
                "arn:aws:s3:::[Source Bucket Name]",
                "arn:aws:s3:::[Source Bucket Name]/*"
            ]
        },
        {
            "Action": [
                "s3:ReplicateObject",
                "s3:ReplicateDelete",
                "s3:ReplicateTags",
                "s3:GetObjectVersionTagging",
                "s3:ObjectOwnerOverrideToBucketOwner"
            ],
            "Effect": "Allow",
            "Resource": "arn:aws:s3:::[Destination Bucket Name]/*"
        }
    ]
}

Destination Bucket에 가서 아래와 같이 설정해준다.

![5](https://user-images.githubusercontent.com/41605276/79063403-87ba3c00-7cdc-11ea-812c-250dc54ce90c.png)

STEP 3) 정상적으로 Replication 설정이 되었는지 테스트

아래 그림과 같이 Source Bucket에 testtest.txt 파일을 업로드 하면 그대로 Destination Bucket에도 똑같은 prefix에 똑같은 파일로 업로드가 된 것을 확인 할 수 있다.

![6](https://user-images.githubusercontent.com/41605276/79063595-9e14c780-7cdd-11ea-85c6-d17e58aa9e67.png)

여기서 테스트를 더 해보면 위에 testtext.txt 파일의 내용이 'version 2'이라는 문자열이 들어가 있는 파일이었는데 내용을 'version 3'로 바꿔서 Source Bucket에 업로드 했을때 똑같이 Destination Bucket에도 반영이 되는지 보자. 아래 그림과 같이 소스버킷의 파일의 내용을 바꿔서 업로드해서 Destination 버킷에도 그대로 반영되는 것을 확인할 수 있다.

![7](https://user-images.githubusercontent.com/41605276/79063804-db2d8980-7cde-11ea-873f-7bb21f624cf8.png)

열어보면 내용이 testtest.txt의 내용이 version 3 인것을 확인할 수 있다.

QA) Souce Bucket의 방금 업로드한 testtest.txt 파일을 삭제하면 Destination Bucket에도 있는 testtest.txt도 지워지는 것인지?

- Answer : 아니다 Destination Bucket에도 있는 testtest.txt은 지워지지 않는다. 위와 같이 똑같은 네임의 객체의 내용이 바뀌어서 새로 업로드 된 경우에는 반영이 되지만 아예 있는 객체를 삭제해도 Destination Bucket의 그 객체도 같이 삭제되지 않는다.

참고자료 : https://docs.aws.amazon.com/ko_kr/AmazonS3/latest/dev/replication-what-is-isnot-replicated.html / S3 개발자 안내서 '삭제 작업이 복제에 미치는 영향'

The official text - "If you specify an object version ID to delete in a DELETE request, Amazon S3 deletes that object version in the source bucket, but it doesn't replicate the deletion in the destination bucket. In other words, it doesn't delete the same object version from the destination bucket. This protects data from malicious deletions. "